# Сравнение предложений
Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:
* кошки (животные)
* UNIX-утилита cat для вывода содержимого файлов
* версии операционной системы OS X, названные в честь семейства кошачьих

Ваша задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.

In [1]:
import re
import numpy as np 
import scipy.spatial as sp

def print_to_file(value, filename):  
    print(value)
    with open(filename, "w") as file:
        file.write(str(value))

Выполните следующие шаги:
1. Скачайте файл с предложениями (sentences.txt).
2. Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().
3. Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.

In [2]:
with open('sentences.txt', "r") as file:
    sentences = np.array(file.readlines())
sentences = map(lambda sentence: sentence.lower(), sentences)
sentences_words = list(list(filter(None, re.split('[^a-z]', sentence))) 
                       for sentence in sentences)
print('Первое предложение, разбитое на слова', sentences_words[0])

Первое предложение, разбитое на слова ['in', 'comparison', 'to', 'dogs', 'cats', 'have', 'not', 'undergone', 'major', 'changes', 'during', 'the', 'domestication', 'process']


4. Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.
5. Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 * 254.

In [3]:
#определим количество уникальных слов
unique_words = set()
for sentence in sentences_words:
    for word in sentence:
        unique_words.add(word)
unique_words_dict = dict()
i=0
for word in unique_words:
    unique_words_dict[word] = i
    i+=1
print ('Количество уникальных слов в корпусе', len(unique_words_dict))

Количество уникальных слов в корпусе 254


In [4]:
sentences_count = len(sentences_words)
unique_words_count = len(unique_words)
bag_of_words = np.zeros((sentences_count, unique_words_count), dtype = int)
i = 0
for sentence in sentences_words:
    for word in sentence:
        if word in unique_words_dict:
            j = unique_words_dict[word]
            bag_of_words[i,j]+=1
    i+=1

6. Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.
7. Запишите полученные числа в файл, разделив пробелом. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-1.txt).

In [5]:
sentence_one = bag_of_words[0]
distances = np.zeros((sentences_count, 2))
for i in np.arange(0, sentences_count):
    distance = sp.distance.cosine(sentence_one, bag_of_words[i])
    distances[i] = [distance, i]
distances = distances[1:]
distances = distances[distances[:,0].argsort()]

In [6]:
result = ' '.join(distances[:2,1:].flatten().astype(int).astype(str))
print_to_file(result, 'submission-1.txt')

6 4


8. Совпадают ли ближайшие два предложения по тематике с первым? Совпадают ли тематики у следующих по близости предложений?

In [7]:
print ('Рейтинг похожести')
print ('Нулевое предложение:', sentences_words[0])
for dist in distances:
    index = int(dist[1])
    sentence = sentences_words[index]
    print (index, ' предложение похоже на первое с такой вероятностью', dist[0], ':', sentence)

Рейтинг похожести
Нулевое предложение: ['in', 'comparison', 'to', 'dogs', 'cats', 'have', 'not', 'undergone', 'major', 'changes', 'during', 'the', 'domestication', 'process']
6  предложение похоже на первое с такой вероятностью 0.7327387580875756 : ['domestic', 'cats', 'are', 'similar', 'in', 'size', 'to', 'the', 'other', 'members', 'of', 'the', 'genus', 'felis', 'typically', 'weighing', 'between', 'and', 'kg', 'and', 'lb']
4  предложение похоже на первое с такой вероятностью 0.7770887149698589 : ['in', 'one', 'people', 'deliberately', 'tamed', 'cats', 'in', 'a', 'process', 'of', 'artificial', 'selection', 'as', 'they', 'were', 'useful', 'predators', 'of', 'vermin']
21  предложение похоже на первое с такой вероятностью 0.8250364469440588 : ['the', 'fifth', 'major', 'update', 'to', 'mac', 'os', 'x', 'leopard', 'contains', 'such', 'a', 'mountain', 'of', 'features', 'more', 'than', 'by', 'apple', 's', 'count']
10  предложение похоже на первое с такой вероятностью 0.8328165362273942 : ['ca

## Выводы вступившего на курс

### Сравнения
* 4 и 6 предложения похожи на нулевое, так как они посвящены кошкам.
* 1 и 2 предложения не похожи на нулевое: нулевое посвящено кошкам, а первое и второе - команде UNIX.
* 21 предложение не похоже на нулевое, хотя и стоит на третьем месте по похожести:  нулевое посвящено кошкам, а 21 - версии операционной системы OS X.

### Итог
Не во всех случаях такой мешок слов смог найти похожие тексты. Разумеется, использованный вами метод крайне простой. Например, он не учитывает формы слов (так, cat и cats он считает разными словами, хотя по сути они означают одно и то же), не удаляет из текстов артикли и прочие ненужные слова. Позже мы будем подробно изучать анализ текстов, где выясним, как достичь высокого качества в задаче поиска похожих предложений.